In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
import pickle
import torchgeometry as tgm

In [2]:
!ls ../blockchain_files/3000000to3999999_BlockTransaction

3000000to3999999_BlockTransaction.csv  graphs_data  processed.csv
extracted_labels.csv		       model.pkl    time.csv
final.csv			       model.pt     tx_counts.csv


In [3]:
 
with open("../blockchain_files/3000000to3999999_BlockTransaction/graphs_data", "rb") as fp:   # Unpickling
    read_graph_data = pickle.load(fp)

In [4]:
data_list = []
count = 0
for data in read_graph_data:
    
    if data.x.shape[0] == 0:
        continue
        
    data_list.append(data)

        
    

In [5]:
len(data_list)

628

In [6]:
train_loader = DataLoader(data_list, batch_size=8,shuffle=True)
test_loader = DataLoader(data_list, batch_size=8,shuffle=False)

In [2]:
from typing import Tuple, Union

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn import BatchNorm1d, Linear

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj, OptTensor, PairTensor


class CGConv(MessagePassing):

    def __init__(self, channels: Union[int, Tuple[int, int]], dim: int = 0,
                 aggr: str = 'mean', batch_norm: bool = False,
                 bias: bool = True, **kwargs):
        super().__init__(aggr=aggr, **kwargs)
        # print(channels)
        self.channels = channels
        self.dim = dim
        self.batch_norm = batch_norm

        if isinstance(channels, int):
            channels = (channels, channels)
        self.lin_f = Linear(2*channels[0] + dim, channels[1], bias=bias)
        # self.lin_s = Linear(2*channels[0] + dim, channels[1], bias=bias)
        if batch_norm:
            self.bn = BatchNorm1d(channels[1])
        else:
            self.bn = None

        self.reset_parameters()


    def reset_parameters(self):
        self.lin_f.reset_parameters()
        # self.lin_s.reset_parameters()
        if self.bn is not None:
            self.bn.reset_parameters()


    def forward(self, x: Union[Tensor, PairTensor], edge_index: Adj,
                edge_attr: OptTensor = None) -> Tensor:
        """"""
        if isinstance(x, Tensor):
            x: PairTensor = (x, x)

        # propagate_type: (x: PairTensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        out = out if self.bn is None else self.bn(out)
        out += x[1]
        # print(out.shape)
        # print(x[1])
        return out


    def message(self, x_i, x_j, edge_attr: OptTensor) -> Tensor:
        if edge_attr is None:
            z = torch.cat([x_i, x_j], dim=-1)
        else:
            z = torch.cat([x_i, x_j, edge_attr], dim=-1)

        # print(z.shape)
        # print(self.lin_f)
        return self.lin_f(z)

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.channels}, dim={self.dim})'

In [3]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
#         torch.manual_seed(12345)
        self.conv1 = CGConv((1, hidden_channels),5,batch_norm=True)
#         self.conv1 = GCNConv(1, hidden_channels)
        
        self.conv2 = CGConv((hidden_channels, hidden_channels),5,batch_norm=True)
        self.conv3 = CGConv((hidden_channels, hidden_channels),5,batch_norm=True)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index,edge_attributes, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index,edge_attributes)
        x = x.relu()
        x = self.conv2(x, edge_index,edge_attributes)
        x = x.relu()
        x = self.conv3(x, edge_index,edge_attributes)
        x = x.relu()
        x = self.conv4(x, edge_index)
 

        # 2. Readout layer
#         print(x.shape)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
#         print(x.shape)
        # 3. Apply a final classifier
#         x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): CGConv((1, 64), dim=5)
  (conv2): CGConv((64, 64), dim=5)
  (conv3): CGConv((64, 64), dim=5)
  (conv4): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [14]:
model = GCN(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# weights = torch.tensor([.05,.95]).to(device)
focal_loss = tgm.losses.FocalLoss(alpha=0.5, gamma=2.0, reduction='mean')

In [15]:
model.conv1.lin_f.weight.device

device(type='cuda', index=0)

In [16]:
device

device(type='cuda')

In [13]:
def train():
    model.train()
    for batch_idx,data in enumerate(train_loader):


        data.edge_attr = data.edge_attr.float()
        data = data.to(device)
        out = model(data.x, data.edge_index,data.edge_attr,data.batch)

        loss = focal_loss(out.reshape(-1,2,1,1), data.y.reshape(-1,1,1))   # Compute the loss.
        loss.backward()  # Derive gradients.
        
        optimizer.step()
        optimizer.zero_grad()
#             print(batch_idx)
        

    return loss.cpu().item()
    
def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        


        data.edge_attr = data.edge_attr.float() 
        data = data.to(device)
        
        out = model(data.x, data.edge_index,data.edge_attr, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        

        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

# test(test_loader)

In [18]:

for epoch in range(1, 401):
    loss = train()
#     break
    train_acc = test(test_loader)
#     test_acc = test(test_loader)
    if epoch % 5 == 0: 
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, loss : {loss:.4f}')

Epoch: 005, Train Acc: 0.7532, loss : 0.0397
Epoch: 010, Train Acc: 0.7197, loss : 0.0194
Epoch: 015, Train Acc: 0.7277, loss : 0.0316
Epoch: 020, Train Acc: 0.7404, loss : 0.0266
Epoch: 025, Train Acc: 0.7420, loss : 0.1511
Epoch: 030, Train Acc: 0.7325, loss : 0.0132
Epoch: 035, Train Acc: 0.7882, loss : 0.0584
Epoch: 040, Train Acc: 0.7277, loss : 0.0450
Epoch: 045, Train Acc: 0.7771, loss : 0.0446
Epoch: 050, Train Acc: 0.7850, loss : 0.1125
Epoch: 055, Train Acc: 0.8010, loss : 0.0352
Epoch: 060, Train Acc: 0.8280, loss : 0.0421
Epoch: 065, Train Acc: 0.7325, loss : 0.0147
Epoch: 070, Train Acc: 0.8392, loss : 0.0872
Epoch: 075, Train Acc: 0.8519, loss : 0.1577
Epoch: 080, Train Acc: 0.7882, loss : 0.0387
Epoch: 085, Train Acc: 0.7818, loss : 0.0384
Epoch: 090, Train Acc: 0.8710, loss : 0.0210
Epoch: 095, Train Acc: 0.8471, loss : 0.0417
Epoch: 100, Train Acc: 0.7914, loss : 0.0240
Epoch: 105, Train Acc: 0.8089, loss : 0.0344
Epoch: 110, Train Acc: 0.8615, loss : 0.0297
Epoch: 115

In [19]:
train_acc = test(test_loader)

In [20]:
train_acc

0.9490445859872612

In [13]:
import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([p.numel() for p in model_parameters])
params

22338

In [22]:
read_graph_data[0]

Data(x=[12995, 1], edge_index=[2, 17027], edge_attr=[17027, 5], y=[1])

In [7]:
path = "../blockchain_files/4000000to4999999_BlockTransaction/graphs/"

In [28]:
with open(path+"graphs_data_0", "rb") as fp:   # Unpickling
    read_graph_data = pickle.load(fp)

In [8]:
import glob

In [9]:
ts_data_list = []
for name in glob.glob(path+'/*'):
#     print(name)
    with open(name, "rb") as fp:   # Unpickling
        read_graph_data = pickle.load(fp)
    if read_graph_data.x.shape[0] == 0:
        continue
    ts_data_list.append(read_graph_data)


In [10]:
len(ts_data_list)

1405

In [11]:
test_loader = DataLoader(ts_data_list, batch_size=2,shuffle=False)

In [14]:
test(test_loader)

0.8355871886120997

In [38]:
PATH = '../blockchain_files/3000000to3999999_BlockTransaction/model.pt'
torch.save(model.state_dict(), PATH)

In [39]:
with open('../blockchain_files/3000000to3999999_BlockTransaction/model.pkl', "wb") as fp:   #Pickling
        pickle.dump(model, fp)

In [4]:
with open('../blockchain_files/3000000to3999999_BlockTransaction/model.pkl', "rb") as fp:   # Unpickling
    model = pickle.load(fp)

In [13]:
test(test_loader)

0.9490445859872612